In [181]:
import h5py
import numpy as np

In [182]:
src_path = "/home/zimgong/Documents/robocasa/datasets/v0.1"
filter_key = "valid"
tgt_path = "datasets/lerobot"
debug = True

task_id = "single_stage/kitchen_coffee/CoffeePressButton"
hdf5_file = f"{src_path}/{task_id}/2024-04-25/demo_gentex_im128_randcams.hdf5"

# task_id = "multi_stage/brewing/PrepareCoffee"
# hdf5_file = f"{src_path}/{task_id}/2024-05-07/demo_im128.hdf5"

dataset_base = f"robocasa/{task_id}"
hdf5_lw = f"/home/zimgong/Downloads/ep_demo.hdf5"

In [ ]:
all_filter_keys = None
f = h5py.File(hdf5_file, "r")
if filter_key is not None:
    # use the demonstrations from the filter key instead
    print("NOTE: using filter key {}".format(filter_key))
    demos = sorted(
        [elem.decode("utf-8") for elem in np.array(f["mask/{}".format(filter_key)])]
    )
else:
    # use all demonstrations
    demos = sorted(list(f["data"].keys()))

    # extract filter key information
    if "mask" in f:
        all_filter_keys = {}
        for fk in f["mask"]:
            fk_demos = sorted(
                [elem.decode("utf-8") for elem in np.array(f["mask/{}".format(fk)])]
            )
            all_filter_keys[fk] = fk_demos

# put demonstration list in increasing episode order
inds = np.argsort([int(elem[5:]) for elem in demos])
demos = [demos[i] for i in inds]

flw = h5py.File(hdf5_lw, "r")
demos_lw = sorted(list(flw["data"].keys()))

In [184]:
demo = demos[0]
demo_lw = demos_lw[0]

In [185]:
demo_ds = f["data/{}".format(demo)]
demo_ds_lw = flw["data/{}".format(demo_lw)]

In [186]:
demo_states = demo_ds["states"]
demo_obs = demo_ds["obs"]

In [ ]:
for key in demo_obs.keys():
    print(key)
    print(demo_obs[key].shape)

In [188]:
def compute_mapping(demo_id):
    demo_ds = f["data/{}".format(demo_id)]
    demo_states = demo_ds["states"]
    print("demo_states", demo_states.shape)
    demo_obs = demo_ds["obs"]
    for key in demo_obs.keys():
        if len(demo_obs[key].shape) == 2:
            for j in range(demo_obs[key].shape[1]):
                if np.allclose(demo_obs[key][:, j], np.zeros(demo_obs[key].shape[0])):
                    print(f"key {key}-{j} invalid")
    for i in range(demo_states.shape[1]):
        if np.allclose(demo_states[:, i], np.zeros(demo_states.shape[0])):
            # print(f"line {i} all zeros, skipped")
            continue
        flag = False
        for key in demo_obs.keys():
            if len(demo_obs[key].shape) == 2:
                for j in range(demo_obs[key].shape[1]):
                    if np.allclose(demo_obs[key][:, j], demo_states[:, i]):
                        flag = True
                        print(f"line {i} key {key}-{j}")
        if not flag:
            print(f"line {i} unmapped")

In [ ]:
compute_mapping(demos[0])

In [ ]:
compute_mapping(demos[1])

In [ ]:
compute_mapping(demos[2])

In [ ]:
# demo_lw_states = demo_ds_lw["states"]
# for i in range(demo_lw_states.shape[1]):
#     if np.allclose(demo_lw_states[:, i], np.zeros(demo_lw_states.shape[0])):
#         # print(f"line {i} all zeros, skipped")
#         continue
#     else:
#         print(f"line {i} valid")